In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
from google.colab import files

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 로컬 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이동
for filename in uploaded.keys():
    # 업로드된 파일을 구글 드라이브의 원하는 경로로 이동
    !mv "{filename}" "/content/drive/MyDrive/{filename}"

Mounted at /content/drive


Saving bs.csv to bs.csv


In [4]:
df = pd.read_csv('/content/drive/MyDrive/bs.csv', header=0, parse_dates=True, encoding='cp949')

In [ ]:
df

,Date,수입(TEU),수출(TEU),수입환적(TEU),수출환적(TEU),수입물가지수,usd비율(Yuan/US),Dubai국제유가(US/배럴),강수량(mm),평균기온,...,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125
0,2014/01,"353,915.75","365,515.00","408,239.00","389,979.75",128.15,6.105,104.0,9.6,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014/02,"334,478.00","334,191.25","313,752.00","350,400.25",128.34,6.121,105.0,44.1,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014/03,"405,896.25","402,860.75","410,435.50","383,563.50",127.66,6.152,104.4,136.7,9.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014/04,"391,681.50","402,606.75","390,004.00","401,684.25",127.56,6.158,104.6,147.8,14.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014/05,"396,793.75","395,188.75","402,278.75","388,978.25",127.70,6.169,105.6,145.9,18.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,2023/03,"493,589.25","465,645.25","550,140.00","540,680.00",121.61,6.869,78.5,76.2,12.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,2023/04,"462,679.00","467,042.00","535,106.75","552,678.25",120.72,6.923,83.4,101.6,14.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,2023/05,"470,223.75","470,912.25","513,907.25","515,848.00",116.48,7.109,75.0,290.7,17.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,2023/06,"438,147.25","447,073.75","501,481.00","497,254.75",115.23,NaN,NaN,153.9,22.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
## 결측치 제거
df = df.dropna(axis = 1, how = 'all')
df = df.dropna()

In [6]:
## Feature Engineering
df['Date'] = pd.to_datetime(df['Date'])
df['연도'] = df['Date'].dt.year
df['월'] = df['Date'].dt.month
df = df.drop(['Date'], axis=1)

In [7]:
## 특정 칼럼에 대해 쉼표 제거 및 숫자로 변환
clms = ['수입(TEU)', '수출(TEU)', '수입환적(TEU)', '수출환적(TEU)']
for clm in clms:
  df[clm] = df[clm].str.replace(',', '').astype(float)
df

,수입(TEU),수출(TEU),수입환적(TEU),수출환적(TEU),수입물가지수,usd비율(Yuan/US),Dubai국제유가(US/배럴),강수량(mm),평균기온,평균최저기온,평균최고기온,연도,월
0,353915.75,365515.00,408239.00,389979.75,128.15,6.105,104.0,9.6,5.0,1.2,9.7,2014,1
1,334478.00,334191.25,313752.00,350400.25,128.34,6.121,105.0,44.1,5.8,2.7,9.9,2014,2
2,405896.25,402860.75,410435.50,383563.50,127.66,6.152,104.4,136.7,9.8,6.2,14.5,2014,3
3,391681.50,402606.75,390004.00,401684.25,127.56,6.158,104.6,147.8,14.6,11.2,19.3,2014,4
4,396793.75,395188.75,402278.75,388978.25,127.70,6.169,105.6,145.9,18.7,15.4,23.2,2014,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,401093.00,417851.25,524469.00,489668.00,123.42,6.753,80.4,79.2,3.8,-0.2,8.9,2023,1
109,410600.50,430718.00,442852.00,471788.00,123.78,6.945,82.1,30.0,6.8,2.9,11.8,2023,2
110,493589.25,465645.25,550140.00,540680.00,121.61,6.869,78.5,76.2,12.3,8.5,17.0,2023,3
111,462679.00,467042.00,535106.75,552678.25,120.72,6.923,83.4,101.6,14.6,11.0,19.0,2023,4


In [8]:
dy = pd.read_csv('/content/drive/MyDrive/부산_해양기상 (1).csv', encoding='cp949')

In [9]:
ds = pd. read_csv('/content/drive/MyDrive/한국_공휴일 (1).csv', encoding='cp949')
ds = ds.drop('locdate', axis=1)

In [10]:
merged_df = pd.merge(dy, ds, on=['연도', '월'])
merged_df

,연도,월,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 수온(°C),평균 유의 파고(m),평균 파주기(sec),한국,중국,일본
0,2014,1,6.253333,1023.482759,52.133333,14.053333,0.976667,6.336667,1,3,2
1,2014,2,7.807143,1022.729630,65.392857,13.289286,1.650000,7.264286,1,2,1
2,2014,3,6.080000,1019.790000,67.133333,13.096667,1.167742,6.425806,1,1,1
3,2014,4,5.116667,1016.806667,72.866667,14.790000,0.877778,5.877778,0,3,1
4,2014,5,5.283333,1012.036667,79.266667,16.340000,0.886667,5.896667,2,5,4
...,...,...,...,...,...,...,...,...,...,...,...
108,2023,1,7.464516,1014.648387,76.193548,23.861290,1.387097,4.706452,5,9,3
109,2023,2,6.096429,1022.039286,63.142857,21.375000,0.892857,4.135714,0,0,2
110,2023,3,6.461290,1020.825806,62.806452,19.183871,0.996774,4.309677,1,1,1
111,2023,4,6.862069,1023.131034,55.827586,16.248276,1.100000,4.531034,0,3,1


In [11]:
df = pd.merge(merged_df, df, on=['연도','월'])
df

,연도,월,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 수온(°C),평균 유의 파고(m),평균 파주기(sec),한국,중국,...,수출(TEU),수입환적(TEU),수출환적(TEU),수입물가지수,usd비율(Yuan/US),Dubai국제유가(US/배럴),강수량(mm),평균기온,평균최저기온,평균최고기온
0,2014,1,6.253333,1023.482759,52.133333,14.053333,0.976667,6.336667,1,3,...,365515.00,408239.00,389979.75,128.15,6.105,104.0,9.6,5.0,1.2,9.7
1,2014,2,7.807143,1022.729630,65.392857,13.289286,1.650000,7.264286,1,2,...,334191.25,313752.00,350400.25,128.34,6.121,105.0,44.1,5.8,2.7,9.9
2,2014,3,6.080000,1019.790000,67.133333,13.096667,1.167742,6.425806,1,1,...,402860.75,410435.50,383563.50,127.66,6.152,104.4,136.7,9.8,6.2,14.5
3,2014,4,5.116667,1016.806667,72.866667,14.790000,0.877778,5.877778,0,3,...,402606.75,390004.00,401684.25,127.56,6.158,104.6,147.8,14.6,11.2,19.3
4,2014,5,5.283333,1012.036667,79.266667,16.340000,0.886667,5.896667,2,5,...,395188.75,402278.75,388978.25,127.70,6.169,105.6,145.9,18.7,15.4,23.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2023,1,7.464516,1014.648387,76.193548,23.861290,1.387097,4.706452,5,9,...,417851.25,524469.00,489668.00,123.42,6.753,80.4,79.2,3.8,-0.2,8.9
109,2023,2,6.096429,1022.039286,63.142857,21.375000,0.892857,4.135714,0,0,...,430718.00,442852.00,471788.00,123.78,6.945,82.1,30.0,6.8,2.9,11.8
110,2023,3,6.461290,1020.825806,62.806452,19.183871,0.996774,4.309677,1,1,...,465645.25,550140.00,540680.00,121.61,6.869,78.5,76.2,12.3,8.5,17.0
111,2023,4,6.862069,1023.131034,55.827586,16.248276,1.100000,4.531034,0,3,...,467042.00,535106.75,552678.25,120.72,6.923,83.4,101.6,14.6,11.0,19.0


In [12]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import copy

In [13]:
class ContainerWaterPrediction:
    def __init__(self, StockData, division_rate=0.7, window_step=1):
        self.m_sData = StockData
        self.m_division_rate = division_rate
        self.m_window_step = window_step
        self.Prepare_Data()

    def Prepare_Data(self):
        self.Setup_Scaler()
        self.Divide_Data()
        self.m_trainX, self.m_trainY = self.Make_Dataset(self.m_TrainData)
        self.m_trainX = np.reshape(self.m_trainX, (self.m_trainX.shape[0], 1, self.m_trainX.shape[1]))
        self.m_testX, self.m_testY = self.Make_Dataset(self.m_TestData)
        self.m_testX = np.reshape(self.m_testX, (self.m_testX.shape[0], 1, self.m_testX.shape[1]))

    def Setup_Scaler(self):
        self.m_Scaler = MinMaxScaler()
        self.m_sDataScaled = self.m_Scaler.fit_transform(self.m_sData.to_numpy().reshape(-1, 1))

    def Divide_Data(self):
        size_sDataScaled = len(self.m_sDataScaled)
        div_pos = int(size_sDataScaled * self.m_division_rate)
        self.m_TrainData = self.m_sDataScaled[0:div_pos, :]
        self.m_TestData = self.m_sDataScaled[div_pos:size_sDataScaled, :]

    def Make_Dataset(self, dataset):
        data_x = []
        data_y = []
        for i in range(len(dataset)- self.m_window_step -1):
            slice = dataset[i:(i+ self.m_window_step), 0]
            data_x.append(slice)
            data_y.append((dataset[i + self.m_window_step], 0))
        data_x = np.array(data_x)
        data_y = np.array(data_y)
        return data_x, data_y

    def Build_ML(self, epoch_val=10):
        self.m_Model = Sequential()
        self.m_Model.add(LSTM(256, input_shape=(1, self.m_window_step)))
        self.m_Model.add(Dense(1))
        self.m_Model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
        self.m_Model.summary()
        self.m_Model.fit(self.m_trainX, self.m_trainY, epochs=epoch_val, batch_size=1, verbose=1)

    def Evaluate_ML(self):
        score = self.m_Model.evaluate(self.m_trainX, self.m_trainY, verbose=0)
        print('Loss = ',score[0])
        print('Accuracy = ',score[1])

    def Predict_ML(self):
        self.m_TrainEstimated = self.m_Model.predict(self.m_trainX)
        self.m_TestEstimated = self.m_Model.predict(self.m_testX)
        self.m_TrainEstimated = self.m_Scaler.inverse_transform(self.m_TrainEstimated)
        self.m_trainY = self.m_Scaler.inverse_transform([self.m_trainY])
        self.m_TestEstimated = self.m_Scaler.inverse_transform(self.m_TestEstimated)
        self.m_testY = self.m_Scaler.inverse_transform([self.m_testY])

    def Generate_Graph(self):
        PlotEstimated = np.empty_like(self.m_sDataScaled)
        PlotEstimated[:, :] = np.nan
        trainPlotEstimated = copy.deepcopy(PlotEstimated)
        trainPlotEstimated[self.m_window_step:len(self.m_TrainEstimated)+self.m_window_step, :] = self.m_TrainEstimated
        testPlotEstimated = copy.deepcopy(PlotEstimated)
        testPlotEstimated[len(self.m_TrainEstimated)+self.m_window_step+2:len(self.m_sDataScaled)-self.m_window_step, :] = self.m_TestEstimated
        plt.plot(self.m_Scaler.inverse_transform(self.m_sDataScaled))
        plt.plot(trainPlotEstimated)
        plt.plot(testPlotEstimated)
        plt.show()

In [14]:
# 데이터 정규화 (0과 1 사이의 값으로 스케일링)
scaler = MinMaxScaler()
all_data_scaled = scaler.fit_transform(df)

In [15]:
#LSTM 모델 구현 및 학습, 예측
ContainerWaterPrediction = ContainerWaterPrediction(df, division_rate=0.7, window_step=3)
ContainerWaterPrediction.Build_ML(10)
ContainerWaterPrediction.Evaluate_ML()
ContainerWaterPrediction.Predict_ML()

<ipython-input-13-4cbe17f455e8>:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_y = np.array(data_y)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               266240    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 266,497
Trainable params: 266,497
Non-trainable params: 0
_________________________________________________________________


ValueError: ignored

In [ ]:
## 전처리
sc = MinMaxScaler()
sc_df = sc.fit_transform(df)

##시퀀스 데이터 생성
sequence_length = 2
sequences = []
target = []

for i in range(len(df) - sequence_length):
    sequences.append(df.iloc[i:i+sequence_length].values)
    target.append(df.iloc[i+sequence_length])

X = np.array(sequences)
y = np.array(target)

## 학습용 데이터와 테스트용 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
## LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## LSTM 모델 구축
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=df.shape[1]))  # 출력 레이어, 컬럼 수에 맞춤

## 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

## 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32)

## 테스트 데이터에 대한 예측
predictions = model.predict(X_test)

## 예측 결과 출력
predicted_df = pd.DataFrame(predictions, columns=df.columns)
print(predicted_df)

Epoch 1/50
3/3 [==============================] - 7s 20ms/step - loss: 34301857792.0000
Epoch 2/50
3/3 [==============================] - 0s 13ms/step - loss: 34301855744.0000
Epoch 3/50
3/3 [==============================] - 0s 13ms/step - loss: 34301849600.0000
Epoch 4/50
3/3 [==============================] - 0s 13ms/step - loss: 34301837312.0000
Epoch 5/50
3/3 [==============================] - 0s 13ms/step - loss: 34301825024.0000
Epoch 6/50
3/3 [==============================] - 0s 14ms/step - loss: 34301810688.0000
Epoch 7/50
3/3 [==============================] - 0s 15ms/step - loss: 34301786112.0000
Epoch 8/50
3/3 [==============================] - 0s 14ms/step - loss: 34301763584.0000
Epoch 9/50
3/3 [==============================] - 0s 14ms/step - loss: 34301726720.0000
Epoch 10/50
3/3 [==============================] - 0s 14ms/step - loss: 34301675520.0000
Epoch 11/50
3/3 [==============================] - 0s 13ms/step - loss: 34301614080.0000
Epoch 12/50
3/3 [=============

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
import os

model = Sequential()
model.add(LSTM(64,
               input_shape=(X_train.shape[1], X_train.shape[2]),
               activation='relu',
               return_sequences=False)
          )
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)

model_path = 'model'
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(X_train, y_train, epochs=200, batch_size=16,
                    validation_data=(X_test, y_test), callbacks=[early_stop, checkpoint])

## 예측 결과 출력
predicted_df = pd.DataFrame(predictions, columns=df.columns)
print(predicted_df)

Epoch 1/200
1/6 [====>.........................] - ETA: 9s - loss: 35260809216.0000
Epoch 1: val_loss improved from inf to 36998156288.00000, saving model to model/tmp_checkpoint.h5
6/6 [==============================] - 2s 67ms/step - loss: 33209233408.0000 - val_loss: 36998156288.0000
Epoch 2/200
1/6 [====>.........................] - ETA: 0s - loss: 31086768128.0000
Epoch 2: val_loss improved from 36998156288.00000 to 36815392768.00000, saving model to model/tmp_checkpoint.h5
6/6 [==============================] - 0s 16ms/step - loss: 31829637120.0000 - val_loss: 36815392768.0000
Epoch 3/200
1/6 [====>.........................] - ETA: 0s - loss: 30513045504.0000
Epoch 3: val_loss did not improve from 36815392768.00000
6/6 [==============================] - 0s 14ms/step - loss: 31438530560.0000 - val_loss: 36883664896.0000
Epoch 4/200
1/6 [====>.........................] - ETA: 0s - loss: 29848027136.0000
Epoch 4: val_loss improved from 36815392768.00000 to 36713037824.00000, saving 

ValueError: ignored

In [ ]:
## 전처리
sc = MinMaxScaler()
sc_df = sc.fit_transform(df)

##시퀀스 데이터 생성
sequence_length = 2
sequences = []
target = []

for i in range(len(df) - sequence_length):
    sequences.append(df.iloc[i:i+sequence_length].values)
    target.append(df.iloc[i+sequence_length])

X = np.array(sequences)
y = np.array(target)

## 학습용 데이터와 테스트용 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
## LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## LSTM 모델 구축
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=df.shape[1]))  # 출력 레이어, 컬럼 수에 맞춤

## 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

## 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32)

## 테스트 데이터에 대한 예측
predictions = model.predict(X_test)

## 예측 결과 출력
predicted_df = pd.DataFrame(predictions, columns=df.columns)
print(predicted_df)

Epoch 1/50
3/3 [==============================] - 8s 18ms/step - loss: 58049228800.0000
Epoch 2/50
3/3 [==============================] - 0s 19ms/step - loss: 58049216512.0000
Epoch 3/50
3/3 [==============================] - 0s 15ms/step - loss: 58049204224.0000
Epoch 4/50
3/3 [==============================] - 0s 18ms/step - loss: 58049179648.0000
Epoch 5/50
3/3 [==============================] - 0s 18ms/step - loss: 58049163264.0000
Epoch 6/50
3/3 [==============================] - 0s 15ms/step - loss: 58049130496.0000
Epoch 7/50
3/3 [==============================] - 0s 15ms/step - loss: 58049089536.0000
Epoch 8/50
3/3 [==============================] - 0s 17ms/step - loss: 58049040384.0000
Epoch 9/50
3/3 [==============================] - 0s 18ms/step - loss: 58048974848.0000
Epoch 10/50
3/3 [==============================] - 0s 17ms/step - loss: 58048892928.0000
Epoch 11/50
3/3 [==============================] - 0s 18ms/step - loss: 58048798720.0000
Epoch 12/50
3/3 [=============

In [ ]:
df = df.drop(['평균 풍속(m/s)','평균 수온(°C)','일본'], axis=1)
df

,연도,월,평균기압(hPa),평균 상대습도(%),평균 유의 파고(m),평균 파주기(sec),한국,중국,수입(TEU),수출(TEU),수입환적(TEU),수출환적(TEU),수입물가지수,usd비율(Yuan/US),Dubai국제유가(US/배럴),강수량(mm),평균기온,평균최저기온,평균최고기온
0,2014,1,1023.482759,52.133333,0.976667,6.336667,1,3,353915.75,365515.00,408239.00,389979.75,128.15,6.105,104.0,9.6,5.0,1.2,9.7
1,2014,2,1022.729630,65.392857,1.650000,7.264286,1,2,334478.00,334191.25,313752.00,350400.25,128.34,6.121,105.0,44.1,5.8,2.7,9.9
2,2014,3,1019.790000,67.133333,1.167742,6.425806,1,1,405896.25,402860.75,410435.50,383563.50,127.66,6.152,104.4,136.7,9.8,6.2,14.5
3,2014,4,1016.806667,72.866667,0.877778,5.877778,0,3,391681.50,402606.75,390004.00,401684.25,127.56,6.158,104.6,147.8,14.6,11.2,19.3
4,2014,5,1012.036667,79.266667,0.886667,5.896667,2,5,396793.75,395188.75,402278.75,388978.25,127.70,6.169,105.6,145.9,18.7,15.4,23.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2023,1,1014.648387,76.193548,1.387097,4.706452,5,9,401093.00,417851.25,524469.00,489668.00,123.42,6.753,80.4,79.2,3.8,-0.2,8.9
109,2023,2,1022.039286,63.142857,0.892857,4.135714,0,0,410600.50,430718.00,442852.00,471788.00,123.78,6.945,82.1,30.0,6.8,2.9,11.8
110,2023,3,1020.825806,62.806452,0.996774,4.309677,1,1,493589.25,465645.25,550140.00,540680.00,121.61,6.869,78.5,76.2,12.3,8.5,17.0
111,2023,4,1023.131034,55.827586,1.100000,4.531034,0,3,462679.00,467042.00,535106.75,552678.25,120.72,6.923,83.4,101.6,14.6,11.0,19.0
